## This code reads the 4D training/testing data arrays, calculates the mean and std values of each of the bands, gets the sample prediction from the trained CNN and saves it all into a Pandas dataframe

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import scipy
from scipy import signal
import xml.etree.ElementTree as ET
from tensorflow.keras.models import load_model

Function that takes the 4D input array normalizes the data and returns the ice/water prediction from the trained CNN

In [ ]:
def class_data(X):
    
    classifier = load_model(r'../../data/SeaIceCNN.h5')

    df = pd.read_pickle(r"../../data/means_stds_S0_cnn.pkl")
    
    for i in range(X.shape[3]):
        
        X[:,:,:,i] = ((X[:,:,:,i] - df.means.iloc[i]) / df.stds.iloc[i]).astype(np.float64)
        
    return np.concatenate(classifier.predict(X, use_multiprocessing=True))

Load training ice data

In [ ]:
icedata_temp = np.load(r"../../data/TrainingDataS0_ice_rev.npy")

Create a dataframe with the mean values of HH, HV, incidence angle and NESZ as well as the standard deviation of HH and HV for each 20x20 pixel subsample

In [ ]:
df = pd.DataFrame({'HH':np.mean(icedata_temp[:,:,:,0], axis=(1,2)),
                  'HV':np.mean(icedata_temp[:,:,:,1], axis=(1,2)),
                  'Angle':np.mean(icedata_temp[:,:,:,2], axis=(1,2)),
                  'NESZ':np.mean(icedata_temp[:,:,:,3], axis=(1,2)),
                  'std_HH':np.std(icedata_temp[:,:,:,0], axis=(1,2)),
                  'std_HV':np.std(icedata_temp[:,:,:,1], axis=(1,2)),
                  'cnn_prediction':class_data(icedata_temp)})

Load test ice data

In [ ]:
icedata_temp = np.load(r"../../data/TestDataS0_ice_rev.npy")

Append the test data to the training data in the dataframe

In [ ]:
df = df.append(pd.DataFrame({'HH':np.mean(icedata_temp[:,:,:,0], axis=(1,2)),
                             'HV':np.mean(icedata_temp[:,:,:,1], axis=(1,2)),
                             'Angle':np.mean(icedata_temp[:,:,:,2], axis=(1,2)),
                             'NESZ':np.mean(icedata_temp[:,:,:,3], axis=(1,2)),
                             'std_HH':np.std(icedata_temp[:,:,:,0], axis=(1,2)),
                             'std_HV':np.std(icedata_temp[:,:,:,1], axis=(1,2)),
                             'cnn_prediction':class_data(icedata_temp)}))

Save the dataframe to a pickle file <br>
Note: This file is included in the shared data on Zenodo

In [ ]:
df.to_pickle(r'../../data/AllIceDF.pkl')

Delete the numpy 4D array

In [ ]:
del icedata_temp

Load the water training data

In [ ]:
waterdata_temp = np.load(r"../../data/TrainingDataS0_water_rev.npy")

Generate the dataframe for the water subsamples

In [ ]:
df = pd.DataFrame({'HH':np.mean(waterdata_temp[:,:,:,0], axis=(1,2)),
                  'HV':np.mean(waterdata_temp[:,:,:,1], axis=(1,2)),
                  'Angle':np.mean(waterdata_temp[:,:,:,2], axis=(1,2)),
                  'NESZ':np.mean(waterdata_temp[:,:,:,3], axis=(1,2)),
                  'std_HH':np.std(waterdata_temp[:,:,:,0], axis=(1,2)),
                  'std_HV':np.std(waterdata_temp[:,:,:,1], axis=(1,2)),
                  'cnn_prediction':class_data(waterdata_temp)})

Load the test water data

In [ ]:
waterdata_temp = np.load(r"../../data/TestDataS0_water_rev.npy")

Append the test data to the training data in the dataframe

In [ ]:
df = df.append(pd.DataFrame({'HH':np.mean(waterdata_temp[:,:,:,0], axis=(1,2)),
                             'HV':np.mean(waterdata_temp[:,:,:,1], axis=(1,2)),
                             'Angle':np.mean(waterdata_temp[:,:,:,2], axis=(1,2)),
                             'NESZ':np.mean(waterdata_temp[:,:,:,3], axis=(1,2)),
                             'std_HH':np.std(waterdata_temp[:,:,:,0], axis=(1,2)),
                             'std_HV':np.std(waterdata_temp[:,:,:,1], axis=(1,2)),
                             'cnn_prediction':class_data(waterdata_temp)}))

Save the dataframe to a pickle file
Note: This file is included in the shared data on Zenodo

In [ ]:
df.to_pickle(r'../../data/AllWaterDF.pkl')

Delete the water 4D numpy array and the temporary dataframe

In [ ]:
del waterdata_temp, df

Load the two dataframes

In [ ]:
dfice = pd.read_pickle(r'../../data/AllIceDF.pkl')
dfwater = pd.read_pickle(r'../../data/AllWaterDF.pkl')

Generate dataframes that are sorted into incidence angle "bins"

In [ ]:
bins = np.arange(19, 51)

wat_ang = dfwater.groupby(pd.cut(dfwater['Angle'],bins=bins))
ice_ang = dfice.groupby(pd.cut(dfice['Angle'],bins=bins))

Create a curve for the NESZ of a SCWA image <br>
Note: This section cannot be reproduced unless you have access to a RSAT-2 image product

In [ ]:
m = ET.parse(r"product.xml")
root = m.getroot()

for lut in root.iter('{http://www.rsi.ca/rs2/prod/xml/schemas}referenceNoiseLevel'):

    if lut.attrib['incidenceAngleCorrection'] == 'Sigma Nought':

        steps = int(lut.findall('{http://www.rsi.ca/rs2/prod/xml/schemas}stepSize')[0].text)
        first_value = int(lut.findall('{http://www.rsi.ca/rs2/prod/xml/schemas}pixelFirstNoiseValue')[0].text)
        noise = np.array(lut.findall('{http://www.rsi.ca/rs2/prod/xml/schemas}noiseLevelValues')[0].text.split(' '),np.float32)

gains_temp = np.zeros(10523, np.float32)
gains_temp[first_value::steps] = np.power(10, noise/10)
kernel = signal.triang(2*steps - 1)
gains_interp = 10 * np.log10(scipy.ndimage.filters.convolve(gains_temp, kernel, mode="constant"))

pref = root.tag.strip('product')
nearang = np.float32(root.find(pref + 'imageGenerationParameters').find(pref + 'sarProcessingInformation').find(pref + 'incidenceAngleNearRange').text)
farang = np.float32(root.find(pref + 'imageGenerationParameters').find(pref + 'sarProcessingInformation').find(pref + 'incidenceAngleFarRange').text)
incang = np.interp(np.arange(10523),[0,len(np.arange(10523))-1],[farang,nearang])

incang_norm = [(x - np.min(incang)) / (np.max(incang) - np.min(incang)) * (len(bins) - 2) + 1 for x in incang]


Create a polynomial curve to fit the datasets (see figures below)

In [ ]:
from scipy.optimize import curve_fit
def func2(x, a, b, c):
    return a*x**2+b*x+c

## Figure: Box plots of the mean HH distributions for all the subsamples

## Results: The polynomial fits with their correlation coefficients are provided

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,10))
bp1 = ax1.boxplot(ice_ang['HH'].unique(), whis = [5, 95], sym = '', 
            labels = bins[0:-1], positions = np.arange(0.8, len(bins)-0.2, 1), 
            widths = 0.3, patch_artist = True)
bp2 = ax1.boxplot(wat_ang['HH'].unique(), whis = [5, 95], sym = '',
            labels = bins[0:-1], positions = np.arange(1.2, len(bins), 1),
            widths = 0.3, patch_artist = True)

ice_hh_med = []
for med in bp1['medians']:
    ice_hh_med.append(med.get_ydata()[0])    
water_hh_med = []
for med in bp2['medians']:
    water_hh_med.append(med.get_ydata()[0])
popt, pcov = curve_fit(func2, bins[:-1], np.array(water_hh_med))
popt2, pcov2 = curve_fit(func2, bins[:-1], np.array(ice_hh_med))

r2_ice=1-np.sum((ice_hh_med-func2(bins[:-1],*popt2))**2)/np.sum((ice_hh_med-np.mean(ice_hh_med))**2)
r2_water=1-np.sum((water_hh_med-func2(bins[:-1],*popt))**2)/np.sum((water_hh_med-np.mean(water_hh_med))**2)

line1 = ax1.plot(np.arange(1.2, len(bins), 1), func2(bins[:-1],*popt), '-b', linewidth=2)
line2 = ax1.plot(np.arange(0.8, len(bins)-.2, 1), func2(bins[:-1],*popt2), '-g', linewidth=2)
line3 = ax1.plot(incang_norm, gains_interp, '-r', linewidth = 3)

for box in bp1['boxes']:
    box.set(facecolor = 'green')
    

for box in bp2['boxes']:
    box.set(facecolor = 'blue')
ax1.set_xlim(0,31)
ax1.set_xticks(np.arange(0, len(bins)+2,2))
ax1.set_xticklabels(np.arange(18, 51,2).tolist(), fontweight = 'bold', fontsize = 22)
ax1.set_ylim(-30,0)
ax1.set_yticklabels(['-30','-25','-20','-15','-10','-5','0'], fontweight = 'bold', fontsize = 22)
ax1.set_ylabel(r'$\sigma^0$ [HH] (dB)', fontsize = 26, fontweight = 'bold')
ax1.set_xlabel(r'Incidence Angle ($^O$)', fontsize = 26, fontweight = 'bold')
ax1.legend([bp1["boxes"][0], bp2["boxes"][0], line3[0]], ['Ice', 'Water', 'NESZ'], loc='upper right', fontsize=26)

print('\u03C3\u00b0[ice] = %5.3f\u03B8\u00b2 %5.3f\u03B8 %5.3f , R\u00b2=%5.3f' % tuple(np.append(popt2,r2_ice)))
print('\u03C3\u00b0[water] = %5.3f\u03B8\u00b2 %5.3f\u03B8 + %5.3f , R\u00b2=%5.3f' % tuple(np.append(popt,r2_water)))

fig.savefig(r'../../figures/Figure3.png')

σ°[ice] = 0.006θ² -0.632θ -4.885 , R²=0.995 <br>
σ°[water] = 0.023θ² -2.263θ + 29.997 , R²=0.999

<center><img src="../../data/figures/Figure3.png" height="500px"></center>

<center>Figure 3. Backscatter intensity (σ0) distribution per degree of incidence angle for the HH channel. Orange line is the median, boxes correspond to the 1st and 3rd quartile and whiskers represent the 5th and 95th percentile. Red line corresponds to the ScanSAR noise floor (NESZ).</center>

## Figure: Same boxplots for the mean HV values of all the subsamples

In [ ]:
fig2, ax2 = plt.subplots(figsize=(15,10))
bp3 = ax2.boxplot(ice_ang.HV.unique(), whis = [5, 95], sym = '', 
            labels = bins[0:-1], positions = np.arange(0.8, len(bins)-0.2, 1), 
            widths = 0.3, patch_artist = True)
bp4 = ax2.boxplot(wat_ang.HV.unique(), whis = [5, 95], sym = '',
            labels = bins[0:-1], positions = np.arange(1.2, len(bins), 1),
            widths = 0.3, patch_artist = True)
l1 = ax2.plot(incang_norm, gains_interp, '-r', linewidth = 3)

for box in bp3['boxes']:
    box.set(facecolor = 'green')
    

for box in bp4['boxes']:
    box.set(facecolor = 'blue')
ax2.set_xlim(0,31)
ax2.set_xticks(np.arange(0, len(bins)+2,2))
ax2.set_xticklabels(np.arange(18, 51,2).tolist(), fontweight = 'bold', fontsize = 22)
ax2.set_ylim(-32,-22)
ax2.set_yticklabels(np.arange(-32, -21, 2).tolist(), fontweight = 'bold', fontsize = 22)
ax2.set_ylabel(r'$\sigma^0$ [HV] (dB)', fontsize = 26, fontweight = 'bold')
ax2.set_xlabel(r'Incidence Angle ($^0$)', fontsize = 26, fontweight = 'bold')
ax2.legend([bp3["boxes"][0], bp4["boxes"][0], l1[0]], ['Ice', 'Water', 'NESZ'], loc='upper right', fontsize=26)
fig.savefig(r'../../figures/Figure4.png')

<center><img src="../../data/figures/Figure4.png" height="500px"></center>

<center>Figure 4. Backscatter intensity (σ0) distribution per degree of incidence angle for the HV channel. Orange line is the median, boxes correspond to the 1st and 3rd quartile and whiskers represent the 5th and 95th percentile. Red line corresponds to the ScanSAR noise floor (NESZ).</center>